# Pandas 数据更改

Pandas 的 `Series` 与 `DataFrame` 对象是可变对象，意味着可以增加、删除、更改对象指向的数据集。与 Numpy 数组相比，`Series` 与 `DataFrame` 对象除了数据之外还包括行索引以及列索引，故 Pandas 数据修改包括如下内容：
- 行列数据的增加、删除、更改
- 行列索引的重命名、顺序调整
- 数据的排序与专制
- 数据清洗，包括缺失值的填充，删除，增加标志位

本节主要介绍`Series` 与 `DataFrame` 对象中数据与索引的更改。

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 数据复制与赋值更改

对于`Series` 与 `DataFrame` 对象，当进行选取、查看或其它操作时会返回一个新对象。与 Numpy 一样，有些是浅拷贝（视图）操作，有些则是深拷贝操作。对于浅拷贝操作，即返回的对象只是复制元数据（行列索引），仍然与原对象共享数据。对浅拷贝的对象进行数据修改，同时也会修改原对象相同内存位置的数据。

例如下面创建一个序列与对象：

In [3]:
# 时间
times = ['6h', '7h', '8h', '11h', '12h', '13h', '18h', '19h', '20h', '21h']
# 餐时标识，顾客人数、平均消费、总消费
meals = np.array(['B', 'B', 'B', 'L', 'L', 'L', 'D', 'D', 'D', ''])
customers = np.array([60, 70, 65, 100, 230, 150, 100, 300, np.nan, 300])
costs = np.array([6.5, 7, 8, 24, 23, 26, 45, 55.5, 45, np.nan])

# 创建 Series，DataFrame对象
customers_series = pd.Series(customers, index=times)
data = {'meals': meals, 'customers': customers, 'costs': costs}
restaurant_frame = pd.DataFrame(data, index=times)

In [ ]:
customers_series
restaurant_frame

使用对象方法 `copy()` 会返回一个深拷贝的对象：

In [ ]:
ser = customers_series.copy()
df = restaurant_frame.copy()

对深拷贝对象进行修改，是不会影响原对象的数据：

In [ ]:
ser['20h'] = 350
print(ser['20h'], customers_series['20h'])

对于`Series` 与 `DataFrarme`对象，可以使用标签（索引）、整数位置、索引对象与布尔数组等多种方式来选取数据。同时，这种方式进行也可以进行赋值操作：

In [ ]:
ser['6h'] = 0
ser

In [ ]:
ser[['13h', '19h', '21h']] = 99.0
ser

In [ ]:
ser.iloc[::2] = np.nan
ser

In [ ]:
df.loc[df.meals == 'B', 'costs'] = 10000
df

In [ ]:
df.iat[0, 1] = 10
df

## 行列增删

### 添加列

使用`df[newcol] = x`赋值语句可以在`DataFrame`对象增加新列。注意如果指定列名已存在，则会更改该列。

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
# 添加总消费列
df['total_costs'] = df['customers'] * df['costs']

除了传入一个序列外，可以赋值标量与数组方式来添加新列，前者相当于使用广播方式赋值，后者长度须与索引长度一致：

In [ ]:
# 添加老板，盈利率
df['boss'] = '老王'
df['gain_rate'] = [0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3]

还可以使用`loc`运算符来添加新列：

In [ ]:
# 添加盈利
df.loc[:, 'gain'] = df['total_costs'] * df['gain_rate']

下面检查添加的结果：

In [ ]:
# 检查结果
df.info()

In [ ]:
df.head()

在使用上述方式添加新列时，都是在列索引尾部追加的。可以使用`insert`方法在指定位置添加一个新列，其使用语法为：
```python
df.insert(loc, column, value, allow_duplicates=False)
```
主要参数：
- `loc`, 列索引位置
- `column`, 列名
- `values`，数据
- `allow_duplicates=False`，是否允许重复插入。缺省是插入已存在列抛出异常。

In [ ]:
# 在指定位置插入列
df.insert(1, 'nonecol', np.nan)
df.head()

### 删除列

有如下方法可以删除列：
- `del`命令，一次只能删除一列
- `df.drop()`方法，可以删除多列
- `df.pop()`方法，删除一列，并返回删除列

在删除列时，如果指定列名不存在，会抛出`KeyError`。

In [ ]:
# 删除一列，如列名不存在则抛出KeyError
del df['nonecol']

使用`df.drop()`会返回删除列后的新对象，原对象本身并没有改变。函数提供`inplace`参数用于指定是否在对象内部进行删除。`df.drop()`可以删除行或列，使用参数`axis`来指定。其使用语法：
```python
df.drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')
```
主要参数
- `labels=None`，名称列表。
- `axis=0`，0表示删除行（缺省），1表示删除列。
- `inplace=False`, False表示原对象自身不删除，True表示原对象自身改变。
- `index`, 用来删除行，等价于（labels, axis=0）。
- `columns`, 用来删除列，等价于（labels, axis=1）。
- `level`，用于层级索引

In [ ]:
# 删除列
df.drop(['boss', 'gain_rate'], axis=1, inplace=True)
df.drop(columns=['total_costs'], inplace=True)
# 检查结果
df.head()

下面使用`df.pop`方法来删除一列：

In [ ]:
gain_series = df.pop('gain')
df.head()

### 添加与删除行

对于`Series`对象，可以使用赋值语句、`loc`运算符等多种方法添加行。对于`DataFrame`对象，可以使用`loc`运算符来添加行，但要注意的是，传入对象必须与列数保持一致。同样，如果指定行标签已存在，会更改该行。

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
df.tail()

In [ ]:
# 添加新行
df.loc['22h'] = ['D', 200, 55]
df.tail()

使用`df.drop()`方法来删除指定行，如果指定行标签不存在会抛出异常。更多内容参见上节：

In [ ]:
# 删除行
df.drop(['22h', '13h'], inplace=True)
df.drop(index=['18h'], inplace=True)
df.tail()

## 索引更改

Pandas 的 `Series` 与 `DataFrame` 对象包含有行与列索引对象。索引对象是不可变的，也就是说不能直接修改或删除索引对象中某个元素。不过作为一个整体，可以进行重命名等操作。

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
df.index
df.columns

In [ ]:
df.index = df.index + '_hour'
df.columns = 'new_' + df.columns
df.head()

### 重命名行列

可以直接使用`df.index`, `df.columns` 的赋值语句来更改行与列索引：

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
df.columns = ['key1', 'key2', 'key3']
df

Pandas 提供了`df.rename()`方法可以同时重命名行与列：
```python
df.rename(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False, level=None)
```
主要参数
- `mapper=None`，映射函数。
- `index`, 用来重命名行，等价于（mapper, axis=0）。
- `columns`, 用来重命名列，等价于（mapper, axis=1）。
- `axis=None`，0表示行（缺省），1表示列。
- `inplace=False`, False表示返回新对象，True表示自身改变。
- `level`，用于层级索引

In [ ]:
df.rename(columns={'key1': '用餐', 'key2': '顾客人数', 'key3': '平均消费'}, inplace=True)
df.head()

除了使用字典外，还可以使用匿名函数来重名行列索引：

In [ ]:
# 重命名 6h => 6hour
df.rename(index=lambda x: x.replace('h', 'hour'))

### 重建索引

Pandas 对象的`reindex`方法使用新的索引对象，将数据进行重新排列，生成一个新对象。如果某个索引不存在，会引入缺失值。对于顺序数据（例如时间序列）等，可以通过参数`method`指定插值方法来填充。其使用语法为：
```python
df.reindex(labels=None, index=None, columns=None, axis=None, method=None, copy=True, level=None, fill_value=nan, limit=None, tolerance=None)
```
主要参数
- `labels=None`，新标签（索引）。
- `index`, 用来重建行索引，等价于（labels, axis=0）。
- `columns`, 用来重建列索引，等价于（labels, axis=1）。
- `axis=None`，0表示行，1表示列。
- `method=None`，填充方法。
- `copy=True`，是否复制新索引。
- `fill_value`：引入缺失值时使用的替代值。

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
newidx = ['6h', '7h', '8h', '9h', '11h', '12h', '13h', '14h', '18h', '19h', '20h', '21h']
newdf = df.reindex(index=newidx)
newdf

`df.reindex`方法支持如下插值方法：
- `ffill`，前向填充
- `bfill`，后向填充

下面语句更改`DataFrame`对象行索引为整数序列，

In [ ]:
# 更改行索引为整数序列：
df.index = [6, 7, 8, 11, 12, 13, 18, 19, 20, 21]
# 使用前向填充
newidx = [6, 7, 8, 9, 11, 12, 13, 14, 18, 19, 20, 21, 22]
newdf = df.reindex(index=newidx, method='ffill', fill_value=10)
newdf

### 索引与列转换

`DataFrame`对象提供如下方法实现索引与列之间的转换：
- `df.reset_index()`把索引转换为列；
- `df.set_index()` 则把列设置为索引；

二者会返回一个新对象，使用参数`inplace=True`则在对象内部进行改变：

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
# 把索引转换为列
df.reset_index(inplace=True)
df

In [ ]:
# 把`index`列转换为索引
df.set_index('index')

在后续章节分层索引中有更多介绍。

### 列格式转换

`Series` 用于同质的数据集，即数据类型都是一样的。`DataFrame` 用于异质的数据集，不同列的数据类型可以不同，但同一列数据类型则是一样的。Pandas 与 Python 内置、Numpy的数据类型对比如下：

| Pandas  |  Python  |  Numpy  |  用途| 
|:-------:|:-------:|:-------:|:-------:|
| object | str | string_, unicode_ | 文本 | 
| int64 | int | int_, int8, int16, int32, int64, uint8, uint16, uint32, uint64 | 整数 | 
| float64 | float | float_, float16, float32, float64 | 浮点数 | 
| bool | bool | bool_ | 布尔数（True/False）  | 
| datetime64 | NA | datetime64[ns] | 日期   |
| timedelta[ns] | NA | NA | 时间间隔  |
| category | NA | NA | 类别数据  |

使用 Pandas 对象的 `astype()`方法，可以转换数据类型。其使用方法为：
```python
df.astype(dtype, copy=True, errors='raise', **kwargs)
s.astype(dtype, copy=True, errors='raise', **kwargs)
```
主要参数为：
- `dtype`，数据类型或字典
- `errors='raise'`，在数据类型转换时会出错，缺省情况出错会抛出异常。设置为`ignore`则忽略。

下面改变一个 `Series` 对象的数据类型，返回一个新对象：

In [20]:
ser = pd.Series([1, 2], dtype='int32')
ser

ser2 = ser.astype('float64')
ser2

0    1
1    2
dtype: int32

0    1.0
1    2.0
dtype: float64

下面转换一个 `DataFrame` 对象的数据格式：

In [21]:
# 深拷贝 restaurant_frame, customers_series
df = restaurant_frame.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 6h to 21h
Data columns (total 3 columns):
meals        10 non-null object
customers    9 non-null float64
costs        9 non-null float64
dtypes: float64(2), object(1)
memory usage: 320.0+ bytes


In [23]:
# 转换 meals列为类别数据类型； customers列为字符串
newdf = df.astype(dtype={'meals': 'category', 'customers': np.str_})
newdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 6h to 21h
Data columns (total 3 columns):
meals        10 non-null category
customers    10 non-null object
costs        9 non-null float64
dtypes: category(1), float64(1), object(1)
memory usage: 442.0+ bytes


## 排序

对数据集进行排序是常用操作。`Series` 与 `DataFrame` 对象包括数据与索引，可以使用索引与列值进行排序。

### 索引排序

Pandas 对象提供 `sort_index()`返回一个排序的新对象，其使用方法为：
```python
df.sort_index(axis=0, level=None, ascending=True, inplace=False, kind='quicksort', na_position='last', sort_remaining=True, by=None)
```
主要参数
- `axis`，0表示行，1表示列
- `ascending=True`，顺序亦或降序

In [ ]:
ser = pd.Series(np.random.rand(4), index=['l3', 'l1', 'l4', 'l0'])
ser

In [ ]:
# 按降序排序
ser.sort_index(ascending=False)

### 列值排序

使用列中的值进行排序，可以使用 `sort_values()` 方法：
```python
df.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
```
主要参数
- `by`，指定列，字符串或列表。
- `axis=0`，0表示行，1表示列。
- `ascending=True`，布尔数或列表，顺序亦或降序。可以使用列表分别指定排序顺序。
- `na_position=last`，对于缺失值来说，其Nan值无法进行比较排序，缺省情况会放置在对象尾部。

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
# `customers`列升序, `costs`降序
df.sort_values(by=['customers', 'costs'], ascending=[True, False], na_position='first')

### 排名

Pandas对象提供方法`rank()`对数据进行排名，其使用语法为：
```python
ser.rank(axis=0, method='average', numeric_only=None, na_option='keep', ascending=True, pct=False)
```
主要参数
- `method='average'`，指定平级时的排名方法。

In [ ]:
ser = pd.Series(np.random.randint(1, 10, size=6), index=['l3', 'l1', 'l4', 'l0', 'l5', 'l2'])
ser

In [ ]:
# 平级时按照先后顺序进行排名
ser.rank(method='first')

## 转置

与 Numpy 二维数组一样，有时需要对行列进行对换，使用`DataFrame`对象的转置属性`df.T`返回转置结果：

In [ ]:
# 深拷贝`restaurant_frame`
df = restaurant_frame.copy()
df.T